In [1]:
import time
import pandas as pd
import numpy as np
import nibabel
from os import listdir
from os.path import isfile, join
from nibabel.affines import apply_affine
from ontology import *
import config
from pathlib import Path

In [2]:
#target is wellid, Allen MNI coordinates, region cover

In [3]:
config.microarrayFolder

'/Users/lfrench/Desktop/results/AlzAllenPET/data/raw/allen_HBA'

In [4]:
#get all the Sample anot files
all_annotations = pd.DataFrame()
donorFolders = [f for f in listdir(config.microarrayFolder) if 'normalized_microarray_donor' in f]
for donorFolder in donorFolders:
    annotation_filename = os.path.join(config.microarrayFolder, donorFolder, "SampleAnnot.csv")
    annotations = pd.read_csv(annotation_filename)
    annotations["donor"] = donorFolder.replace("normalized_microarray_donor", "")
    #subset to just well ID and MNI coordinates
    #annotations= annotations[["well_id", "structure_id", "mni_x","mni_y","mni_z"]]
    all_annotations = pd.concat([all_annotations,annotations])

In [5]:
all_annotations

,structure_id,slab_num,well_id,slab_type,structure_acronym,structure_name,polygon_id,mri_voxel_x,mri_voxel_y,mri_voxel_z,mni_x,mni_y,mni_z,donor
0,4322,10,5705,CX,Cl,"claustrum, left",978619,120,91,71,-29.2,5.8,-2.6,10021
1,4291,10,5713,CX,Acb,"nucleus accumbens, left",998603,103,96,71,-10.1,5.9,-8.4,10021
2,4292,10,5721,CX,Acb,"nucleus accumbens, right",998604,86,95,69,8.9,8.5,-7.4,10021
3,4292,11,5729,CX,Acb,"nucleus accumbens, right",999200,87,95,76,7.9,0.8,-6.7,10021
4,4314,11,5737,CX,SI,"substantia innominata, right",992030,79,97,76,16.9,0.9,-9.1,10021
5,4323,11,5745,CX,Cl,"claustrum, right",992127,61,86,78,37.0,-0.5,4.1,10021
6,4298,11,5753,CX,GPe,"globus pallidus, external segment, right",991186,82,89,75,13.4,2.2,0.2,10021
7,4382,11,5666,CX,LA,"lateral nucleus, right",992123,72,108,75,24.8,1.7,-22.1,10021
8,4335,11,5674,CX,ATZ,"amygdalohippocampal transition zone, right",992126,77,102,76,19.2,0.7,-14.9,10021
9,4346,11,5682,CX,BLA,"basolateral nucleus, right",992121,78,108,75,18.1,1.6,-22.1,10021


In [6]:
all_annotations = all_annotations.rename(columns={'mni_x' : 'Allen_mni_x', 'mni_y' : 'Allen_mni_y', 'mni_z' : 'Allen_mni_z'})

In [7]:
#lookup MNI values in PET maps

In [8]:
all_annotations.head()

,structure_id,slab_num,well_id,slab_type,structure_acronym,structure_name,polygon_id,mri_voxel_x,mri_voxel_y,mri_voxel_z,Allen_mni_x,Allen_mni_y,Allen_mni_z,donor
0,4322,10,5705,CX,Cl,"claustrum, left",978619,120,91,71,-29.2,5.8,-2.6,10021
1,4291,10,5713,CX,Acb,"nucleus accumbens, left",998603,103,96,71,-10.1,5.9,-8.4,10021
2,4292,10,5721,CX,Acb,"nucleus accumbens, right",998604,86,95,69,8.9,8.5,-7.4,10021
3,4292,11,5729,CX,Acb,"nucleus accumbens, right",999200,87,95,76,7.9,0.8,-6.7,10021
4,4314,11,5737,CX,SI,"substantia innominata, right",992030,79,97,76,16.9,0.9,-9.1,10021


In [9]:
def get_nifti_val_truncating(row, sform_matrix, img, col_prefix):
    """
    Function for apply method that returns the corresponding NIFTI values from
    the input MNI coordinates. Function should not be used for anything except for apply.
    :param row: the tuple/row that is passed in from the apply method
    :return: NIFTI value in new space
    """
    new_coords = apply_affine(sform_matrix, [int(row[col_prefix + '_mni_x']), int(row[col_prefix + '_mni_y']), int(row[col_prefix + '_mni_z'])])
    nifti_value = img.get_data()[int(new_coords[0]), int(new_coords[1]), int(new_coords[2])]
    #One map returns singletons
    if isinstance(nifti_value, np.memmap):
        nifti_value = nifti_value[0]
    return nifti_value



In [12]:
#Schwartz PET study
img = nibabel.load(config.DebNIFTIfile)
sform_matrix = img.header.get_sform()
# invert the sform transformation so we can start with MNI coordinates
sform_matrix = np.linalg.inv(sform_matrix)
# get the value of the white/black gradient from Deborah Schwartz's meta analysis - data in the 0 index
mni_maps = ["Allen"]
for mapping in mni_maps:
    all_annotations[mapping + "_Schwartz_NiftiValue"] = all_annotations.apply(get_nifti_val_truncating, axis=1, args=(sform_matrix, img, mapping))


In [13]:
def get_reg_of_interest(ontology_obj, reg_spec, reg_children=None, seen=None):
    """
    Returns None if whole_brain is chosen and reg_spec[0] is None. Otherwise returns
    a list of the regions of interest, according to the first item in
    region_specification tuple, and removing any of those listed for exclusion
    as the second item of the tuple. This currently has not been coded to handle
    multiple regions of inclusions, so this function would need to be run for
    each region of inclusion that is desired.
    :param ontology_obj: an ontology object
    :param reg_spec: tuple of str and str of region specifications
           where the str indicates a region id of inclusion, and the
           second is a list of str indicating region ids of exclusion
    :param reg_children: used for recursion, it accumulates the regions that
           are to be included
    :param seen: used for recursion, it tracks regions that have already been
           added or excluded from reg_children
    :return: set of int (structure_ids)
    """
    if reg_spec[0] == None or (reg_spec[1] and reg_spec[0] in reg_spec[1]):
        return None
    if not reg_children:
        reg_children = set()

    if not seen:
        seen = set()

    seen.add(int(reg_spec[0]))
    reg_children.add(int(reg_spec[0]))

    if ontology_obj.hierarchy.get(int(reg_spec[0])):
        curr_children_set = {x for x in ontology_obj.hierarchy.get(int(reg_spec[0]))}
        reg_children = reg_children.union(curr_children_set)
        for child in curr_children_set:
            if child not in seen:
                seen.add(child)
                if (reg_spec[1]):
                    if str(child) not in reg_spec[1]:
                        reg_children = reg_children.union(get_reg_of_interest(ontology_obj, (str(child), reg_spec[1]),
                                                                              reg_children, seen))
                    else:
                        reg_children.discard(child)
                else:
                    reg_children = reg_children.union(get_reg_of_interest(ontology_obj, (str(child), reg_spec[1]),
                                                                          reg_children, seen))
    return reg_children


In [14]:
#handle region classifications
REG_OF_INTEREST_DICT = {
                        "cortex": ('4008', None),
                        "cortex_excluding_piriform_hippocampus": ('4008', ['4249', '10142'])}
CURR_REGION_OF_INTEREST = "cortex_excluding_piriform_hippocampus"
ontology_obj = Ontology(os.path.join(".", "data", "Ontology.csv"))
regions_of_interest = get_reg_of_interest(ontology_obj, REG_OF_INTEREST_DICT[CURR_REGION_OF_INTEREST])


In [15]:
for CURR_REGION_OF_INTEREST in REG_OF_INTEREST_DICT:
    print(CURR_REGION_OF_INTEREST)
    ontology_obj = Ontology(os.path.join(".", "data", "Ontology.csv"))
    regions_of_interest = get_reg_of_interest(ontology_obj, REG_OF_INTEREST_DICT[CURR_REGION_OF_INTEREST])
    all_annotations["in_"+CURR_REGION_OF_INTEREST] = all_annotations.structure_id.isin(regions_of_interest)

cortex
cortex_excluding_piriform_hippocampus


In [16]:
all_annotations.head()

,structure_id,slab_num,well_id,slab_type,structure_acronym,structure_name,polygon_id,mri_voxel_x,mri_voxel_y,mri_voxel_z,Allen_mni_x,Allen_mni_y,Allen_mni_z,donor,Allen_Schwartz_NiftiValue,in_cortex,in_cortex_excluding_piriform_hippocampus
0,4322,10,5705,CX,Cl,"claustrum, left",978619,120,91,71,-29.2,5.8,-2.6,10021,0.000100,False,False
1,4291,10,5713,CX,Acb,"nucleus accumbens, left",998603,103,96,71,-10.1,5.9,-8.4,10021,0.000069,False,False
2,4292,10,5721,CX,Acb,"nucleus accumbens, right",998604,86,95,69,8.9,8.5,-7.4,10021,0.000894,False,False
3,4292,11,5729,CX,Acb,"nucleus accumbens, right",999200,87,95,76,7.9,0.8,-6.7,10021,0.000673,False,False
4,4314,11,5737,CX,SI,"substantia innominata, right",992030,79,97,76,16.9,0.9,-9.1,10021,0.000107,False,False


In [17]:
#write out combined csv file
all_annotations.to_csv("./data/Combined_SampleAnnot_with_PET.csv", index=False)